In [ ]:
import boto3
import json

In [ ]:
REGION = "us-east-1"
KNOWLEDGE_BASE_ID = "A3XBZT3D8Y"

In [ ]:
bedrock_agent = boto3.client('bedrock-agent-runtime',region_name=REGION)


In [ ]:
def retrieve_context(query,kb_id,top_k=5):
    response = bedrock_agent.retrieve(
        knowledgeBaseId=kb_id,
        retrievalQuery={"text": query},
        retrievalConfiguration={
            "vectorSearchConfiguration": {
                "numberOfResults": top_k
            }
        }
    )
    contexts = []
    for r in response.get("retrievalResults",[]):
        contexts.append({
            "text": r.get("content", {}).get("text", ""),
            "score": r.get("score"),
            "metadata": r.get("metadata")
        })
    return [response,contexts]

In [20]:
myquery = "Which patients have had a heart attack in the past?"
k = 10
rsp,context = retrieve_context(myquery,KNOWLEDGE_BASE_ID,k)
context[0]

{'text': 'Discharge Summary\r Document ID: MED-20230515-001 Date Created: May 15, 2023 --- Patient Information Patient ID: PT-789012 Name: Jane Doe Date of Birth: November 23, 1978 (45 years old) Gender: Female Address: 456 Oak Avenue, Anytown, CA 90210 Contact Number: 555-987-6543 --- Clinician Information Clinician ID: CLI-54321 Name: Dr. Emily White Specialization: Cardiology Institution: City General Hospital Contact Email: emily.white@citygeneral.com --- Chief Complaint Acute chest pain and shortness of breath --- History of Present Illness Patient is a 45-year-old female presenting to the ER with sudden onset of severe, crushing chest pain radiating to the left arm, accompanied by dyspnea, nausea, and diaphoresis for the past 2 hours. Symptoms started at rest. No prior history of similar episodes. Denies recent trauma or fever. --- Past Medical History Conditions: Hypertension (diagnosed 5 years ago, on Lisinopril), Hyperlipidemia (diagnosed 2 years ago, on Atorvastatin). Allergi

In [24]:
import os
os.path.basename(context[0]['metadata']['x-amz-bedrock-kb-source-uri'])

'medical_document_1035.md'